In [1]:
# 1. Import libraries
# Core libraries
import pandas as pd
import numpy as np
# Machine learning - Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import (
    RandomForestRegressor,
    BaggingRegressor,
    GradientBoostingRegressor,
    ExtraTreesRegressor
)
from sklearn.neural_network import MLPRegressor
# Other ML libraries
import xgboost as xgb
from catboost import CatBoostRegressor
# Deep learning - TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
# Plotting
import matplotlib.pyplot as plt

In [2]:
# 2. Load dataset
#df = pd.read_csv("pure state for 5.csv") 
J1=0.5
df = pd.read_csv("Pure states training for "+str(J1)+".csv") 
df=df.dropna()
X = df.drop(columns=["N"])
y = df["N"]

In [3]:
# Explor The Data
print("Data shape:",df.shape)
print("X shape:",X.shape)
print("y shape:",y.shape)

Data shape: (9999, 6)
X shape: (9999, 5)
y shape: (9999,)


In [4]:
# 3. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# 4. Scale features (for NN)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_scaled = scaler.transform(X)

In [ ]:
# 5. Build and train Neural Network
nn_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

nn_model.compile(optimizer='adam', loss='mse')

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

nn_history = nn_model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=128,
    callbacks=[early_stop],
    verbose=0
)


C:\Users\ahmed167010\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Plot training and validation loss from the history
plt.figure(figsize=(8, 5))
plt.plot(nn_history.history['loss'], label='Training Loss')
plt.plot(nn_history.history['val_loss'], label='Validation Loss')
plt.title('Neural Network - Epochs vs Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# 6. Train XGBoost model
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=200,
    learning_rate=0.05,
    max_depth=5,
    random_state=42
)

xgb_model.fit(X_train, y_train)

In [ ]:
# 7. Train ُExtra Trees model
et_model = ExtraTreesRegressor(
    n_estimators=200,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)
et_model.fit(X_train, y_train)

In [ ]:
# 8. Predict with both models
nn_pred = nn_model.predict(X_test_scaled).flatten()
xgb_pred = xgb_model.predict(X_test)
et_pred = et_model.predict(X_test)

In [ ]:
# 10. Evaluate models
def evaluate_model(y_true, y_pred, name="Model"):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{name} Performance:")
    print(f"  MSE: {mse:.6f}")
    print(f"  MAE: {mae:.6f}")
    print(f"  R²: {r2:.6f}\n")

evaluate_model(y_test, nn_pred, "Neural Network")
evaluate_model(y_test, xgb_pred, "XGBoost")
evaluate_model(y_test, et_pred, "Extra Trees")

In [ ]:
# Integrating predictions as inputs to the meta-learner
meta_X = np.column_stack((nn_pred, xgb_pred, et_pred))
meta_y = y_test
# Tain the meta-learner (GradientBoostingRegressor)
#model=MLPRegressor(activation='relu', hidden_layer_sizes=(100,100), max_iter=2000, random_state=42)
#model=RandomForestRegressor(n_estimators=100, random_state=42)
model=CatBoostRegressor(
    n_estimators=1000,
    early_stopping_rounds=100,
    random_state=42,
    verbose=100
)
#model=GradientBoostingRegressor(n_estimators=100, random_state=42)
#model=ExtraTreesRegressor(max_depth=20, max_features=1.0, min_samples_leaf=1, n_estimators=300, random_state=42)
#model=LinearRegression()
meta_learner =model 
meta_learner.fit(meta_X, meta_y)

# Predict the final results
final_preds = meta_learner.predict(meta_X)

# Calculate the metrics (R^2, MSE, MAE)
r2 = r2_score(meta_y, final_preds)
mse = mean_squared_error(meta_y, final_preds)
mae = mean_absolute_error(meta_y, final_preds)

# Print metrics results
print(f"R^2 Score of Stacking Model: {r2}")
print(f"MSE of Stacking Model: {mse}")
print(f"MAE of Stacking Model: {mae}")

In [ ]:

plt.figure(figsize=(6,6))
plt.scatter(y_test, final_preds, alpha=0.3, color='red',s=5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'b-', lw=1.5,label=r"$\bar{\mathcal{N}}$"+r"$=\mathcal{N}$")
x1=y_test.min(); x2=y_test.max(); y1=final_preds.min(); y2=final_preds.max();
m=round((y2-y1)/(x2-x1),3); c=round(y1-m*x1,3);
plt.plot([y_test.min(), y_test.max()], [final_preds.min(), final_preds.max()], 'k--', lw=1.5,\
         label=r"$\bar{\mathcal{N}}= $"+str(m)+ r"$ \mathcal{N} $"+str(c))
plt.xlabel(r"$\mathcal{N}$",fontsize=20)
plt.ylabel(r"$\bar{\mathcal{N}}$",fontsize=20)
plt.title(r"(b)",fontsize=20)
plt.legend(fontsize=18)
plt.grid(True)
plt.axis("equal")
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.ylim(0,1)
plt.tight_layout()
#plt.savefig("pure_state_one_meta.png", dpi=800,bbox_inches='tight')
plt.show()

In [ ]:
############################################################################################################3
##############################################################################################################
##############################################################################################################

In [ ]:

# 8. Predict with both models
nn_p = nn_model.predict(X_scaled).flatten()
xgb_p = xgb_model.predict(X)
et_p = et_model.predict(X)
# Integrating predictions as inputs to the meta-learner
meta_X = np.column_stack((nn_p, xgb_p, et_p))
meta_y = y

# Tain the meta-learner (GradientBoostingRegressor)
meta_learner = model

meta_learner.fit(meta_X, meta_y)


In [ ]:
#The predicting for special class of states

In [ ]:
tested_data=pd.read_csv("Pure states tested for "+str(J1)+".csv")  

In [ ]:

X=tested_data.drop('N',axis=1)
y=tested_data['N']

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_scaled = scaler.transform(X)

In [ ]:
# 8. Predict with both models
nn_X = nn_model.predict(X_scaled).flatten()
xgb_X = xgb_model.predict(X)
et_X = et_model.predict(X)

In [ ]:

# Integrating predictions as inputs to the meta-learner
meta_X_t = np.column_stack((nn_X, xgb_X, et_X))
meta_learner.fit(meta_X_t,y)
# Predict the final results
y_pred = meta_learner.predict(meta_X_t)


In [ ]:
number_iteration=len(y_pred)
α=np.linspace(0,np.pi/2,number_iteration)
plt.plot(α,tested_data['N'],color='blue',label="Exact for "+r"$J=0.5$",linestyle='-')
plt.plot(α,y_pred,color='red',label='Predicted for '+r"$J=0.5$",linestyle='--')
# Set x-axis ticks at special points
special_points = [0, np.pi/8, np.pi/4, 3*np.pi/8, np.pi/2]
labels = [r"$0$", r"$\frac{\pi}{8}$", r"$\frac{\pi}{4}$", r"$\frac{3\pi}{8}$", r"$\frac{\pi}{2}$"]
plt.title(r'(e)',fontsize=20)
plt.xticks(special_points, labels)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.ylim(0,1)
plt.ylabel(r"$\mathcal{N}$",fontsize=20)
plt.legend(fontsize=18)
plt.tight_layout()
#plt.savefig("pure_state_one_tested.png", dpi=800,bbox_inches='tight')
plt.show()